# Recipe: Get schedules for a whole year

## Problem

Get schedules for a whole year.

## Solution

In [1]:
import datetime as dt

import pandas as pd

from cro.schedule import Client, Schedule

In [2]:
YEAR = 2022
month_dates = [dt.date(YEAR, month, 1) for month in range(1, 3)]
data: dict[Schedule, pd.DataFrame] = {}

client = Client()  # Set the station id (sid) later within the for loop.

Fetch the sechedules for station Plus and Radiožurnál from the beginning of the year. 

In [3]:
from tqdm import tqdm

for sid in ("plus", "radiozurnal"):
    client.station = sid
    for date in tqdm(month_dates):
        schedules = client.get_month_schedule(date)
        for schedule in schedules:
            data[schedule] = schedule.to_table()

100%|██████████| 2/2 [00:06<00:00,  3.24s/it]


### Write single dataset to Excel

In [4]:
for schedule, table in tqdm(data.items()):
    week_number = f"{schedule.date.isocalendar()[1]:02d}"
    week_start = schedule.date - dt.timedelta(days=schedule.date.weekday())  # Monday
    week_end = week_start + dt.timedelta(days=6)  # Sunday

    with pd.ExcelWriter(
        f"../../../data/sheet/{YEAR}/Schedule_{schedule.station.name}_{YEAR}W{week_number}_{week_start}_{week_end}.xlsx"
    ) as writer:
        table.to_excel(writer, index=False)

100%|██████████| 118/118 [00:14<00:00,  7.89it/s]


### Write concatenated datasets to Excel

In [5]:
with pd.ExcelWriter(f"../../../data/sheet/Schedule_Y{YEAR}.xlsx") as writer:
    pd.concat(data.values()).to_excel(writer)